# Store Sales - Time Series Forecasting
Predicts the unit sales for thousands of items sold at different Favorita stores. Datatypes of dates, store, and item information, promotions, and unit sales are given.

In [138]:
import pandas as pd # linear algebra
import numpy as np # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visuals
import seaborn as sns # visuals
from warnings import simplefilter

simplefilter("ignore")  # ignore warnings to clean up output cells



train = pd.read_csv('train.csv',index_col=['date'],parse_dates=['date'], infer_datetime_format=True)
test = pd.read_csv('test.csv')

# importing additional data
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
transactions = pd.read_csv('transactions.csv')
holidays = pd.read_csv('holidays_events.csv')


In [139]:
sales = train.copy()
sales = sales.to_period('D')

sales.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [140]:
avg_sales = sales.groupby('date')['sales'].mean()

In [141]:
avg_sales['time'] = np.arange(len(avg_sales.index))
avg_sales.head()

date
2013-01-01      1.409438
2013-01-02    278.390807
2013-01-03    202.840197
2013-01-04    198.911154
2013-01-05    267.873244
Name: sales, dtype: object

In [146]:
from sklearn.linear_model import LinearRegression

# Training data
X = avg_sales.loc[:,'time']  # features
y = avg_sales.loc[:, 'sales']  # target

#df.loc(axis=0)[:, :, 'C1', :]

# Train the model
model = LinearRegression()
model.fit(X, y)

# Store the fitted values as a time series with the same time index as
# the training data
y_pred = pd.Series(model.predict(X), index=X.index)


IndexingError: Too many indexers